# Create an index of the dishes in my scrapbook. 
#### Using a grid


In [1]:
%load_ext blackcellmagic
# use %%black in cell to perform magic

In [1]:
import os
import pickle
import time
#from collections import Counter

# for printing
import tempfile
import win32api
import win32print

# Import tkinter and ttk for UI handling
from tkinter import *
from tkinter import ttk
from tkinter import messagebox

# nltk for singular / plural options
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download("wordnet")
wnl = WordNetLemmatizer()

# method of getting the name of a variable
def get_variable_name(*variable):
    """gets string of variable name
    inputs
        variable (str)
    returns
        string
    """
    if len(variable) != 1:
        raise Exception("len of variables inputed must be 1")
    try:
        return [k for k, v in locals().items() if v is variable[0]][0]
    except:
        return [k for k, v in globals().items() if v is variable[0]][0]


# Does a scrapbook already exist? If so load it. If not start one.
sb_name = "recipies.pkl"
sb_path = os.path.join("cook", sb_name)

if os.path.exists(sb_path):
    f = open(sb_path, "rb")
    recipes = pickle.load(f)
    f.close()
    print("Opened:", sb_path)
else:
    print("No scrapbook exists - creating one in /cook folder")
    recipes = {}  # a directory
    f = open(sb_path, "wb")
    pickle.dump(recipes, f)
    f.close()

print(
    "The current scrapbook >",
    str(get_variable_name(recipes)),
    "< contains:",
    len(recipes),
    "items",
)

# import key ingredients to build a recipe from or to look up a receipe
# See if one exists to import otherwise create a default one

ki_name = "ingredients.pkl"
ki_path = os.path.join("cook", ki_name)

if os.path.exists(ki_path):
    f = open(ki_path, "rb")
    ingredients_list = pickle.load(f)
    f.close()
    print("Opened:", ki_path)
else:
    print("No ingredients list exists - creating one in /cook folder")
    ingredients_list = [
        "apple",
        "beetroot",
        "broad bean",
        "butternut squash",
        "cabbage",
        "carrot",
        "cauliflower",
        "chickpea",
        "clementine",
        "coconut",
        "courgette",
        "egg",
        "ginger",
        "green beans",
        "kohlrabi",
        "nut",
        "onion",
        "parsnip",
        "poori",
        "potato",
        "quorn",
        "rhubarb",
        "tofu",
        "tomatoes",
    ]

# sort the ingredients and save them (### do this as the save point at the end too)
ingredients_list_s = sorted(ingredients_list)

f = open(ki_path, "wb")
pickle.dump(ingredients_list_s, f)
f.close()

print("Sorted Ingredients:", ingredients_list_s)  # to recreate sorted list

# ================================= Now build the main UI ==========================================


# a variety of font sets for buttons, lists, etc.
fontlabl = ("Courier", 14)  # general button
fontlablb = ("Courier", 14, "bold")  # general button bold
fontlabli = ("Courier", 14, "italic")  # general button for an unfinished function
fontlabls = ("Arial", 12)  # smaller for ingredients list
line_length = 70  # so we can pad out things on the printer
new_line = "\n"
tab = "\t"

root = Tk()  # establish a root
root.option_add("*tearOff", FALSE)  # recommended not to allow tear-off menus

# lift window to the front
root.lift()
root.attributes("-topmost", True)

root.title("Recipe Manager")
# aim at upper right side of window (-10 from right, 40 down from top)
root.geometry("-10+40")

new_title = ""
new_course = ""
new_book = ""
new_page = 0
reslist = list()

# ============= define general purpose sub-routines

# error window popup
def err_win(errmsg, parent=root):
    error_win = Toplevel()
    error_win.title("Error report")
    # error_win.geometry("150x30-60+45")
    error_win.geometry("-60+45")
    if parent == "":
        error_win.lift(root)
    else:
        error_win.lift(parent)
    lab = Label(error_win, text=errmsg, width=40, fg="red")
    lab.config(font=fontlablb)
    lab.grid(row=0, column=0, padx=5, pady=5)
    btn_er = Button(error_win, text="OK", command=error_win.destroy)
    btn_er.grid(row=1, column=0, padx=5, pady=5)
    return


# file saver for updated recipe or ingredients
def save_file(path, file):
    f = open(path, "wb")
    pickle.dump(file, f)
    f.close()
    return


# general file print (basic Windows call)
def print_file(fn):
    hinstance = 0
    hinstance = win32api.ShellExecute(
        0, "print", fn, '"%s"' % win32print.GetDefaultPrinter(), ".", 0
    )
    return hinstance


# add a little delay loop to let temp files close (unlock) properly before trying to remove
def remove_file(path, retries=3, sleep=0.1):
    for i in range(retries):
        try:
            os.remove(path)
        except WindowsError:
            time.sleep(sleep)
        else:
            break


# ============= define sub-routines for ingredients


def i_select():  # function to retrieve ingredients selected in main window
    # TODO > look at binding as an option to avoid this button
    # see: https://stackoverflow.com/questions/6554805/getting-a-callback-when-a-tkinter-listbox-selection-is-changed/12936031#12936031

    global reslist  # otherwise it disappears when window closes
    reslist = list()
    # clear the output box (if needed) as now using ingredient selection
    text_log.delete(
        1.0, END,
    )

    selection = lstbox.curselection()
    for i in selection:
        item = lstbox.get(i)
        reslist.append(item)
        msg = "Selected ingredients: " + str(reslist) + new_line
    text_log.insert(END, msg)
# end of confirming ingredients


def i_add():  # add a new ingredient
    def i_add_ok():  # if this isn't in the same 'namespace' as i_add it cannot do the .get on i_new

        new_i = ""
        # all ingredients are to be kept lower case to avoid confusion
        new_i = i_new.get().lower()
        if new_i == "":
            # call common error popup subroutine
            err_win("No ingredient entered", i_win)
        else:
            # check it is not already in the list
            if new_i in ingredients_list_s:
                err_msg = str(new_i) + " is already in the list"
                err_win(err_msg, i_win)
                i_new.delete(0, END)  # clear the entry box
                return
            ingredients_list_s.append(new_i)  # add to end of list
            ingredients_list_s.sort()  # sort in place
            ingredients.set(ingredients_list_s)  # update the listbox
            save_file(ki_path, ingredients_list_s)  # save to disc
            i_new.delete(0, END)  # clear the entry box
            result = new_i + " added" + new_line
            text_log.insert(END, result)
        return

    def i_add_close():  # clean up
        text_log.delete(1.0, END)  # clear the message box
        i_win.destroy()
        return

    i_win = Toplevel()
    i_win.title("Add ingredient")
    i_win.geometry("-60+45")
    i_win.lift(root)  # on top of main app window
    i_lab1 = Label(i_win, text="Enter an ingredient")
    i_lab1.config(font=fontlablb)
    i_lab1.grid(row=0, column=0, columnspan=2, padx=5, pady=5)
    i_new = Entry(i_win, width=60)
    i_new.focus_force()  # make it the cursor focus
    i_new.grid(row=1, column=0, columnspan=2, padx=5, pady=5)
    i_lab2 = Label(i_win, text="Add, or Close to quit")
    i_lab2.grid(row=2, column=0, columnspan=2, padx=5, pady=5)
    i_btn_ok = Button(i_win, text="Add", command=i_add_ok)
    i_btn_ok.config(font=fontlabl)
    i_btn_ok.grid(row=3, column=0, padx=5, pady=5)
    i_btn_c = Button(i_win, text="Close", command=i_add_close)
    i_btn_c.config(font=fontlabl)
    i_btn_c.grid(row=3, column=1, padx=5, pady=5)
    return
# end of add new ingredient

# function to retrieve ingredient selected for deletion
def i_del_select():
    del_i = ""
    selection = id_lstbox.curselection()
    if len(selection) == 1:
        del_i = id_lstbox.get(selection)  # can only be one item in first position

        # double check
        msg = (
            "Are you sure you want to remove: "
            + str(del_i)
            + " from the ingredients list?"
        )
        check = messagebox.askyesnocancel("Confirn", msg)

        if check is True:
            # remove from list, no need to re-sort as only one item
            ingredients_list_s.remove(del_i)
            # update the listbox 'ingredients' file in this window
            ingredientd.set(ingredients_list_s)
            # update the listbox 'ingredients' file in the main waindow
            ingredients.set(ingredients_list_s)
            # and save to disc
            save_file(ki_path, ingredients_list_s)
            msg = del_i + " removed" + new_line
            text_log.insert(END, msg)
            i_win.destroy()
            return
    else:
        # call common error popup subroutine
        err_win("No ingredient selected", i_win)
        i_win.destroy()
    i_win.destroy()
    return

# remove an ingredient ( TODO: check for usage in existing recipies?)
def i_del():
    global i_win
    i_win = Toplevel()
    i_win.title("Remove an ingredient")
    i_win.geometry("-60+45")
    i_win.lift(root)  # on top of main app window

    i_lab1 = Label(i_win, text="Pick an ingredient to remove")
    i_lab1.config(font=fontlablb)
    i_lab1.grid(row=0, column=0, columnspan=2, padx=5, pady=5)
    # add ingredients to a list box

    global id_lstbox, ingredientd  # so it can pass to sibling routine
    ingredientd = StringVar(
        value=ingredients_list_s
    )  # pull in sorted ingredients list into a set

    # only single selections mode here
    id_lstbox = Listbox(
        i_win,
        listvariable=ingredientd,
        selectmode=BROWSE,
        width=25,
        height=40,
        font=fontlabls,
    )
    id_lstbox.grid(row=1, column=0, columnspan=2, padx=5, pady=5)
    i_lab2 = Label(i_win, text="Remove, or Close to quit")
    i_lab2.grid(row=2, column=0, columnspan=2, padx=5, pady=5)
    i_btn_ok = Button(i_win, text="Remove", command=i_del_select)
    i_btn_ok.config(font=fontlabl)
    i_btn_ok.grid(row=3, column=0, padx=5, pady=5)
    i_btn_c = Button(i_win, text="Close", command=i_win.destroy)
    i_btn_c.config(font=fontlabl)
    i_btn_c.grid(row=3, column=1, padx=5, pady=5)
    return

# ============= define sub-routines for recipes

# add new recipe
def new_recipe():
    def retrieve_title():  # if this isn't in the same 'namespace' it cannot reference the newtitle entry widget

        global new_title, new_course, new_book, new_page

        new_title = ""
        new_title = newtitle.get()
        if new_title == "":
            err_win("No title entered", window2)

        new_course = ""
        new_course = newcourse.get()

        new_book = ""
        new_book = newbook.get().upper()
        if new_book == "" or new_book not in ["A", "B"]:
            err_win("No book, or unknown book, entered", window2)
            return

        new_page = IntVar(0)
        try:
            new_page = int(newpage.get())
        except ValueError:
            err_win("Page number is not an integer", window2)
            return
        if new_page < 1:
            err_win("Page number not in range 1-200", window2)
            return
        """       
        new_page = 0
        new_page = newpage.get()
        if new_page == 0:
            err_win("No valid page entered", window2)
            return
        """
        # Add the new recipe if all is well
        if new_title != "":
            if new_title in recipes:
                msg = (
                    "A recipe for "
                    + new_title
                    + " is already entered - not overwriting"
                )
                err_win(msg)
                text_log.insert(END, msg)
            else:
                # add new recipe details
                recipes[new_title] = [reslist, new_course, new_book, new_page]
                # save recipes
                save_file(sb_path, recipes)
                newtitle.delete(0, END)  # clear the entry box
                newcourse.delete(0, END)  # clear the entry box
                newbook.delete(0, END)  # clear the entry box
                newpage.delete(0, END)  # clear the entry box

                msg = (
                    "Added: "
                    + new_title
                    + " on "
                    + str(new_book)
                    + ","
                    + str(new_page)
                    + new_line
                )
                text_log.insert(END, msg)

        # reset
        window2.destroy()
        return new_title, new_course, new_book, new_page

    if reslist == list():
        # call common error popup subroutine
        err_win("No ingredients confirmed yet")
        return

    window2 = Toplevel()
    window2.geometry("-60+45")
    window2.title("Adding New Recipe")
    window2.lift(root)
    newname = Label(window2, text="Add a new recipe/course/ book & page")
    newname.grid(row=0, column=0, columnspan=2, padx=5, pady=5)

    titllabel = Label(window2, text="Enter a title")
    titllabel.config(font=fontlabl)
    titllabel.grid(row=1, column=0, columnspan=2, padx=5, pady=5)
    newtitle = Entry(window2, width=60)
    newtitle.focus_force()  # position the cursor ready to type
    newtitle.grid(row=2, column=0, columnspan=2, padx=5, pady=5)

    corslabel = Label(window2, text="Pick a course (optional)")
    corslabel.config(font=fontlabl)
    corslabel.grid(row=3, column=0, columnspan=2, padx=5, pady=5)
    courses = ["Starter", "Salad", "Main", "Desert"]
    newcourse = ttk.Combobox(window2, values=courses)
    newcourse.grid(row=4, column=0, columnspan=2, padx=5, pady=5)

    booklabel = Label(window2, text="Which recipe book?")
    booklabel.config(font=fontlabl)
    booklabel.grid(row=5, column=0, columnspan=2, padx=5, pady=5)
    books = ["A", "B"]
    newbook = ttk.Combobox(window2, values=books)
    # newbook.current(0)   # set openning choice position not helpful as it still needs to be picked
    newbook.grid(row=6, column=0, columnspan=2, padx=5, pady=5)

    pagelabel = Label(window2, text="What page?")
    pagelabel.config(font=fontlabl)
    pagelabel.grid(row=7, column=0, columnspan=2, padx=5, pady=5)
    newpage = Spinbox(window2, from_=1, to=100)
    newpage.grid(row=8, column=0, columnspan=2, padx=5, pady=5)

    btn_nr = Button(window2, text="Enter", command=retrieve_title)
    btn_nr.grid(row=9, column=0, padx=5, pady=5)

    btnx = Button(window2, text="Close", command=window2.destroy)
    btnx.grid(row=9, column=1, padx=5, pady=5)

    return
# end of add new recipe

# find recipe
def find_recipe():
    text_log.delete(
        1.0, END,
    )  # empty log window
    found = False

    if reslist == []:
        # call common error popup subroutine
        err_win("No ingredients confirmed")
        return

    f, filename = tempfile.mkstemp(".txt")
    pf = open(filename, "w+")
    pf.write("Search results\n")
    
    # TODO - new method of counting multiple hits
    found_recipes = []  # temp list for search findings
    found_items = [] # temp list if multiple items match in same recipie
    
    for r in recipes:
        # search for ingredient keywords
        for item in reslist:
            contents = recipes.get(r)
            # get ingredients list (first item in each directory)
            ingredients = contents[0]
            course = contents[1]
            book = contents[2]
            page = contents[3]
            
            if item in ingredients:
                found = True
                found_items.append(item)
                
        if found:
            tot_items = len(found_items)             
            found_recipes.append([tot_items, r, tuple(found_items), course, book, page])
            found_items.clear() # reset temp list for next recipe
            found = False

    if len(found_recipes) > 0:
        # sort by most matching items (position 0)
        found_recipes_s = sorted(found_recipes, key=lambda x:x[0], reverse=True)
        found_recipes.clear()  # reset temporary list
        
        for i in range(len(found_recipes_s)): 
            print(found_recipes_s[i])
            recipe = found_recipes_s[i][1]
            found_ingredients = found_recipes_s[i][2]
            course = found_recipes_s[i][3]
            book = found_recipes_s[i][4]
            page = found_recipes_s[i][5]
           
            result = (
                "Found a recipe with: "
                + str(found_ingredients)
                + " called "
                + recipe
                + "("
                + course
                + ") in "
                + book
                + "-"
                + str(page)
                + new_line
                )
            print(result)
            text_log.insert(END, result)
            pf.write(result) 
            
    else: # nothing found
        result = "No recipes found for: " + str(reslist)
        text_log.insert(END, result)
        pf.close()
        return
        
    def p_win_ok():
        pf.close()
        result_code = print_file(filename)

    def p_win_end():
        pf.close()
        remove_file(filename)
        p_win.destroy()

    p_win = Toplevel()
    p_win.title("Print report?")
    p_win.geometry("300x100-60+45")
    p_win.lift(root)
    btn_ok = Button(p_win, text="OK", command=p_win_ok)
    btn_ok.grid(row=0, column=0, padx=5, pady=5)
    btn_c = Button(p_win, text="Close when print is finished", command=p_win_end)
    btn_c.grid(row=1, column=0, padx=5, pady=5)
    return


# End of find recipe

# Print a book index
def print_index():
    # store index in a list of lists to allow for sorting before printing.
    book_index = []

    # get  a temporary output file ready (use a known filetype to avoid printing issues)
    f, pi_file = tempfile.mkstemp(".txt")
    pif = open(f, "w+")

    pi_win = Toplevel()
    pi_win.title("Print a scrapbook index")
    pi_win.geometry("-60+45")  # bigger window to accomodate index
    pi_win.lift(root)  # on top of main app window

    pi_lab1 = Label(pi_win, text="Pick a scrapbook to display")
    pi_lab1.config(font=fontlablb)
    pi_lab1.grid(row=0, column=0, columnspan=2, padx=10, pady=10)

    pi_b = StringVar(value="A")  # set a default
    sb_a = Radiobutton(pi_win, text="A", variable=pi_b, value="A")
    sb_a.grid(row=1, column=0, padx=5, pady=5)

    sb_b = Radiobutton(pi_win, text="B", variable=pi_b, value="B")
    sb_b.grid(row=1, column=1, padx=5, pady=5)

    def get_recipes():
        # get recipes and display
        pi_book = pi_b.get()

        gr_win = Toplevel()
        # gr_win.geometry("600x500-60+45")  # bigger window to accomodate index
        gr_win.geometry("-60+45")  #
        gr_win.lift(pi_win)  # on top of parent window
        gr_title = "Index of scrapbook " + str(pi_book)
        gr_win.title(gr_title)

        chars = len(pi_book) + 19
        spacer = " " * ((line_length - chars) // 2)  # to centre
        gr_title = spacer + "Index of scrapbook " + str(pi_book) + new_line
        pif.write(gr_title)

        gr_out = Text(gr_win, height=50, width=75, bg="light cyan")
        gr_out.grid(row=0, column=0, padx=5, pady=5)

        gr_out.insert(END, gr_title)
        gr_out.insert(END, new_line)

        # print header line

        p_line = "Title(Course) " + "Page" + new_line
        gr_out.insert(END, p_line)

        chars = 18
        spacer = "." * (line_length - chars)
        p_linep = "Title(Course) " + spacer + "Page" + new_line
        pif.write(p_linep)

        # scan recipies for chosen book
        for r_title in recipes:
            contents = recipes.get(r_title)
            pi_line = ""

            if contents[2] == pi_book:  # if the book matches
                course = contents[1]
                page = int(
                    contents[3]
                )  # it comes as a string, need integer for sorting
                book_index.append([r_title, course, page])

        # sort recipies by page (position 3)
        book_index_s = sorted(book_index, key=lambda x: x[2])

        for i in book_index_s:

            pi_line = i[0] + "(" + str(i[1]) + ")" + str(i[2]) + new_line
            gr_out.insert(END, pi_line)
            # add padding for printed version
            chars = len(i[0]) + len(i[1]) + len(str(i[2])) + 2
            spacer = "." * (line_length - chars)
            pi_line = i[0] + "(" + str(i[1]) + ")" + spacer + str(i[2]) + new_line
            pif.write(pi_line)

        if pi_line == "":  # nothing found?
            gr_out.insert(END, "No recipes found")

        def gr_ok():
            pif.close()
            result_code = print_file(pi_file)
            gr_win.destroy()
            pi_win.destroy()

        def gr_end():
            pif.close()  # make sure it's closed
            remove_file(pi_file)
            gr_win.destroy()
            pi_win.destroy()

        gr_btn_ok = Button(gr_win, text="Print results", command=gr_ok)
        gr_btn_ok.grid(row=1, column=0, padx=5, pady=5)
        gr_btn_c = Button(gr_win, text="Close when print is finished", command=gr_end)
        gr_btn_c.grid(row=2, column=0, padx=5, pady=5)

    pi_btn_ok = Button(pi_win, text="OK", command=get_recipes)
    pi_btn_ok.grid(row=3, column=0, padx=5, pady=5)
    pi_btn_c = Button(pi_win, text="Cancel", command=pi_win.destroy)
    pi_btn_c.grid(row=3, column=1, padx=5, pady=5)
    return
# End of Print a book index

# Edit or delete a recipe
def edit_recipe():
    # store index in a list of lists to allow for sorting before printing.
    book_index = []

    er_win = Toplevel()
    er_win.title("Edit a recipe")
    er_win.geometry("-60+45")
    er_win.lift(root)  # on top of main app window

    er_lab1 = Label(er_win, text="Pick a scrapbook to display")
    er_lab1.config(font=fontlablb)
    er_lab1.grid(row=0, column=0, columnspan=2, padx=5, pady=5)

    er_book = StringVar(value="A")  # set a default
    er_a = Radiobutton(er_win, text="A", variable=er_book, value="A")
    er_a.grid(row=1, column=0, padx=5, pady=5)
    er_b = Radiobutton(er_win, text="B", variable=er_book, value="B")
    er_b.grid(row=1, column=1, padx=5, pady=5)

    def get_recipes():
        # get the picked book
        global er_bookp
        er_bookp = er_book.get()

        # scan recipes for chosen book
        for r_title in recipes:
            contents = recipes.get(r_title)
            if contents[2] == er_bookp:  # if the book matches
                r_ingredients = contents[0]
                course = contents[1]
                # 'page' may come as a string, need integer for sorting
                page = int(contents[3])
                book_index.append([r_title, course, r_ingredients, page])

        # sort recipies by page (position 3)
        global book_index_s
        book_index_s = sorted(book_index, key=lambda x: x[3])

        # call new window to present recipe list
        pic_recipe()

    def pic_recipe():
        def delete_a_recipe():
            selection = id_lstbox.curselection()
            sel_r = id_lstbox.get(selection)
            title = sel_r[0]
            # double check
            msg = (
                "Are you sure you want to remove: "
                + str(title)
                + " from the recipe list?"
            )
            check = messagebox.askyesnocancel("Confirn", msg)

            if check is True:
                if title in recipes:
                    deleted = recipes.pop(title, "none found")
                    msg = "'" + title + "' has been deleted" + new_line
                    text_log.insert(END, msg)
                    save_file(sb_path, recipes)
                    pr_win.destroy()
                    er_win.destroy()
                    return
                else:
                    # call common error popup subroutine
                    err_win("No matching recipe found", pr_win)
                    pr_win.destroy()
                    er_win.destroy()
                    return

            pr_win.destroy()
            er_win.destroy()
            return

        def edit_a_recipe():
            def update_a_recipe():
                global book_index_s
                new_title = ""
                new_title = newtitle.get()
                if new_title == "":
                    err_win("No title entered", window2)
                    return

                new_course = ""
                new_course = newcourse.get()
                # no need to check if blank as that is an option

                same_ingredients = r_ingredients  # not changed

                new_page = IntVar(0)
                try:
                    new_page = int(newpage.get())
                except ValueError:
                    err_win("Page number is not an integer", window2)
                    return
                if new_page < 1:
                    err_win("Page number not in range 1-200", window2)
                    return

                same_book = er_bookp  # not changed

                # Remove old title then add new title (as title may be the same but different page/course)
                if title in recipes:
                    deleted = recipes.pop(title, "none found")
                    msg = (
                        "Old recipe: '"
                        + str(title)
                        + str(deleted)
                        + "' has been removed"
                        + new_line
                    )
                    text_log.insert(END, msg)
                    save_file(sb_path, recipes)  # save master list
                if new_title in recipes:
                    msg = (
                        "This title already exists elsewhere: ' "
                        + new_title
                        + "' - aborting"
                    )
                    err_win(msg, window2)
                    # restore the old entry
                    recipes[title] = [same_ingredients, course, same_book, int(page)]
                    # save master list
                    save_file(sb_path, recipes)
                    msg = "'" + title + "' has been restored" + new_line
                    text_log.insert(END, msg)
                    # shut windows
                    window2.destroy()
                    pr_win.destroy()
                    er_win.destroy()
                    return
                else:
                    recipes[new_title] = [
                        same_ingredients,
                        new_course,
                        same_book,
                        int(new_page),
                    ]
                    # update master list
                    save_file(sb_path, recipes)
                    msg = "Updated recipe'" + new_title + "' has been added" + new_line
                    text_log.insert(END, msg)
                    ### close this window and parent windows
                    window2.destroy()
                    pr_win.destroy()
                    er_win.destroy()
                    return
                # end of update_a_recipe

            sel_r = ""
            selection = id_lstbox.curselection()
            if len(selection) == 1:
                # really can only be one item in first position
                sel_r = id_lstbox.get(selection)
                # for update and delete subroutines as these are not catured again there
                global title, course, r_ingredients, page
                title = sel_r[0]
                course = sel_r[1]
                # ingredient arrives as a tuple and needs to be a list for listbox
                r_ingredients = list(sel_r[2])
                page = int(sel_r[3])
                # now pop up a new window like the 'add recipe' one, but prepopulated

                window2 = Toplevel()
                window2.geometry("-60+45")
                window2.lift(er_win)
                newname = Label(window2, text="Edit a recipe title, course or page")
                newname.config(font=fontlablb)
                newname.grid(row=0, column=0, columnspan=2, padx=10, pady=5)

                titllabel = Label(window2, text="Title")
                titllabel.config(font=fontlabl)
                titllabel.grid(row=1, column=0, columnspan=2, padx=5, pady=5)
                global newtitle
                newtitle = Entry(window2, width=60)
                newtitle.insert(0, title)  # add title
                newtitle.focus_force()  # position the cursor ready to type
                newtitle.grid(row=2, column=0, columnspan=2, padx=5, pady=5)

                corslabel = Label(window2, text="Course (optional)")
                corslabel.config(font=fontlabl)
                corslabel.grid(row=3, column=0, columnspan=2, padx=5, pady=5)
                courses = ["Starter", "Salad", "Main", "Desert"]
                global newcourse
                newcourse = ttk.Combobox(window2, values=courses)
                if course == "Starter":
                    newcourse.current(0)
                elif course == "Salad":
                    newcourse.current(1)
                elif course == "Main":
                    newcourse.current(2)
                elif course == "Desert":
                    newcourse.current(3)
                newcourse.grid(row=4, column=0, columnspan=2, padx=5, pady=5)

                ingrlabel = Label(window2, text="Ingredients (display only)")
                ingrlabel.config(font=fontlabl)
                ingrlabel.grid(row=5, column=0, columnspan=2, padx=5, pady=5)
                ingrlabel = Label(
                    window2, text="(Recreate recipe to change ingredients)"
                )
                ingrlabel.config(font=fontlabli)
                ingrlabel.grid(row=6, column=0, columnspan=2, padx=5, pady=5)

                ingredientd = StringVar(value=r_ingredients)
                lstbox_ri = Listbox(
                    window2,
                    listvariable=ingredientd,
                    selectmode=BROWSE,
                    width=20,
                    height=10,
                    font=fontlabli,
                )
                lstbox_ri.grid(row=7, column=0, columnspan=2, padx=5, pady=5)

                pagelabel = Label(window2, text="Page")
                pagelabel.config(font=fontlabl)
                pagelabel.grid(row=8, column=0, columnspan=2, padx=5, pady=5)
                page_var = IntVar(value=page)
                global newpage
                newpage = IntVar(0)
                newpage = Spinbox(window2, from_=1, to=200, textvariable=page_var)
                newpage.grid(row=9, column=0, columnspan=2, padx=5, pady=5)

                btn_rr = Button(window2, text="Update recipe", command=update_a_recipe)
                btn_rr.config(font=fontlabl)
                btn_rr.grid(row=10, column=0, padx=5, pady=5)

                btnx = Button(window2, text="Close/Cancel", command=window2.destroy)
                btnx.config(font=fontlabl)
                btnx.grid(row=10, column=1, padx=5, pady=5)
            else:
                # call common error popup subroutine
                err_win("No recipe selected", pr_win)
            return

        # end of edit_recipe

        pr_win = Toplevel()
        pr_win.geometry("-60+45")
        pr_win.lift(er_win)  # on top of parent window
        pr_title = "Index of scrapbook " + str(er_bookp)
        pr_win.title(pr_title)

        # pull in sorted ingredients list into a set
        recipe_list = StringVar(value=book_index_s)

        # only single selections mode here
        id_lstbox = Listbox(
            pr_win,
            listvariable=recipe_list,
            selectmode=BROWSE,
            width=75,
            height=35,
            font=fontlabls,
        )
        id_lstbox.grid(row=1, column=0, columnspan=3, padx=5, pady=5)

        pr_btn_ok = Button(pr_win, text="Edit selected recipe", command=edit_a_recipe)
        pr_btn_ok.config(font=fontlabl)
        pr_btn_ok.grid(row=2, column=0)

        pr_btn_ok = Button(
            pr_win, text="Delete selected recipe", command=delete_a_recipe
        )
        pr_btn_ok.config(font=fontlabl)
        pr_btn_ok.grid(row=2, column=1)

        pr_btn_end = Button(pr_win, text="Close", command=pr_win.destroy)
        pr_btn_end.config(font=fontlabl)
        pr_btn_end.grid(row=2, column=2)

    # end of pic_recipe

    er_btn_ok = Button(er_win, text="OK", command=get_recipes)
    er_btn_ok.grid(row=2, column=0, padx=5, pady=5)
    er_btn_c = Button(er_win, text="Close", command=er_win.destroy)
    er_btn_c.grid(row=2, column=1, padx=5, pady=5)
    return
# end of edit recipe subroutine

# start of search titles with keywords
def find_recipe_title():
    text_log.delete(
        1.0, END,
    )  # empty log window

    def search_title():
        kw_found = False

        key_words = frt_win_e1.get().lower().split(" ")
        key_word_list = []
        for key_word in key_words:
            key_word_list.append(key_word)
            key_word_list.append(wnl.lemmatize(key_word))

        # TODO - prioritise titles with most keywords?
        for t in recipes:
            title = t.lower()  # ignore case
            title_words = title.split(" ")  # seperate the words by comma
            for word in title_words:
                if wnl.lemmatize(word) in key_word_list:
                    kw_found = True
                    # get recipe details
                    contents = recipes.get(t)
                    ingredients = contents[0]
                    course = contents[1]
                    book = contents[2]
                    page = contents[3]
                    msg = (
                        "Found: '"
                        + t
                        + "' in book "
                        + book
                        + ".p"
                        + str(page)
                        + new_line
                    )
                    text_log.insert(END, msg)
        if kw_found is False:
            msg = "Nothing found for keyword(s) '" + str(key_word_list) + "'" + new_line
            text_log.insert(END, msg)

        frt_win.destroy()
        return

    frt_win = Toplevel()
    frt_win.lift(root)
    frt_win.title("Search recipe titles")
    frt_win.geometry("-60+45")
    frt_win_title = Label(frt_win, text="Search titles for keyword")
    frt_win_title.config(font=fontlablb)
    frt_win_title.grid(row=0, column=0, columnspan=2, padx=5, pady=5)

    frt_win_l1 = Label(frt_win, text="Enter a keyword")
    frt_win_l1.config(font=fontlabl)
    frt_win_l1.grid(row=1, column=0, columnspan=2, padx=5, pady=5)
    frt_win_e1 = Entry(frt_win, width=60)
    frt_win_e1.focus_force()  # position the cursor ready to type
    frt_win_e1.grid(row=2, column=0, columnspan=2, padx=5, pady=5)

    frt_win_b1 = Button(frt_win, text="Search", command=search_title)
    frt_win_b1.config(font=fontlabl)
    frt_win_b1.grid(row=3, column=0, padx=5, pady=5)

    frt_win_b2 = Button(frt_win, text="Exit", command=frt_win.destroy)
    frt_win_b2.config(font=fontlabl)
    frt_win_b2.grid(row=3, column=1, padx=5, pady=5)
    return
# end of search titles with keywords

# ============================ end of function definitions ====================
# Main Window Layout

# Left hand side
i_lab = Label(root, text="Ingredients")
i_lab.config(font=fontlablb)
i_lab.grid(row=0, column=0, columnspan=2, padx=10, pady=10)

ai_btn = Button(root, text="Add", command=i_add)
ai_btn.config(font=fontlabl)
ai_btn.grid(row=1, column=0, padx=5, pady=5)

ri_btn = Button(root, text="Remove", command=i_del)
ri_btn.config(font=fontlabl)
ri_btn.grid(row=1, column=1, padx=5, pady=5)

# add ingredients to a list box
ingredients = StringVar(value=ingredients_list_s)

scroll_bar = Scrollbar(root, orient=VERTICAL, bg="light cyan")
scroll_bar.grid(row=2, rowspan=9, column=2, sticky=N + S + W)

lstbox = Listbox(
    root,
    listvariable=ingredients,
    selectmode=EXTENDED,
    height=40,
    font=fontlabls,
    yscrollcommand=scroll_bar.set,
    bg="light cyan",
)
lstbox.grid(row=2, rowspan=9, column=0, columnspan=2, padx=10, pady=10)
# add scrollbar
scroll_bar.config(command=lstbox.yview)

# TODO > look at binding as an option to avoid this button
# see: https://stackoverflow.com/questions/6554805/getting-a-callback-when-a-tkinter-listbox-selection-is-changed/12936031#12936031
""" 
eg. 

def onselect(evt):
    # Note here that Tkinter passes an event object to onselect()
    w = evt.widget
    index = w.curselection()[0]
    value = w.get(index)
    print 'You selected item %d: "%s"' % (index, value)

lb = Listbox(frame, name='lb')
lb.bind('<<ListboxSelect>>', onselect)
"""
# middle (also used by scrollbar for ingredients)
m_lab = Label(root, text="    ")
m_lab.config(font=fontlablb)
m_lab.grid(row=0, column=2, padx=10, pady=10)


# Right hand side
a_lab = Label(root, text="Actions")
a_lab.config(font=fontlablb)
a_lab.grid(row=0, column=3, padx=10, pady=10)

# must do this every time a selection is made
btnc = Button(root, text="Confirm ingredient selection", command=i_select)
btnc.config(font=fontlablb)
btnc.grid(row=1, column=3, padx=5, pady=5)

btna = Button(root, text="Add a new recipe with these ingredients", command=new_recipe)
btna.config(font=fontlabl)
btna.grid(row=2, column=3, padx=5, pady=5)

btnf = Button(root, text="Find a recipe with these ingredients", command=find_recipe)
btnf.config(font=fontlabl)
btnf.grid(row=3, column=3, padx=5, pady=5)

btntf = Button(
    root, text="Search recipe titles for keywords", command=find_recipe_title
)
btntf.config(font=fontlabl)
btntf.grid(row=4, column=3, padx=5, pady=5)

l_lab = Label(root, text="Log")
l_lab.config(font=fontlablb)
l_lab.grid(row=5, column=3, padx=10, pady=10)

text_log = Text(root, height=20, width=50, bg="light cyan")
text_log.grid(row=6, column=3, padx=10, pady=10)

u_lab = Label(root, text="Utilities")
u_lab.config(font=fontlablb)
u_lab.grid(row=7, column=3, padx=10, pady=10)

btni = Button(root, text="Print a book's index", command=print_index)
btni.config(font=fontlabl)
btni.grid(row=8, column=3, padx=5, pady=5)

btne = Button(root, text="Edit a recipe", command=edit_recipe)
btne.config(font=fontlabl)
btne.grid(row=9, column=3, padx=5, pady=5)

btnx = Button(root, text="Exit", command=root.destroy)
btnx.config(font=fontlabl)
btnx.grid(row=10, column=3, padx=5, pady=5)

r11_lab = Label(root, text="Spare row 11")
r11_lab.config(font=fontlabli)
r11_lab.grid(row=11, column=3, padx=5, pady=5)

root.mainloop()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jedba\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Opened: cook\recipies.pkl
The current scrapbook > recipes < contains: 166 items
Opened: cook\ingredients.pkl
Sorted Ingredients: ['almond', 'apple', 'apricot', 'artichoke', 'asparagus', 'aubergine', 'avocado', 'baguette', 'banana', 'barley', 'basmati rice', 'beansprouts', 'beetroot', 'black beans', 'black-eye bean', 'blueberry', 'bread', 'broad bean', 'broad bean tops', 'broccoli', 'brussels sprouts', 'buckwheat', 'bulgur wheat', 'butter beans', 'butternut squash', 'cabbage', 'cannellini beans', 'carrot', 'cashew nuts', 'cauliflower', 'celeriac', 'chard', 'cheese', 'chickpea', 'chickpea flour', 'chilli', 'chocolate', 'clementine', 'coconut', 'corn', 'cottage cheese', 'courgette', 'cranberries', 'cucumber', 'currents', 'custard', 'dates', 'double cream', 'edamame beans', 'egg', 'fennel', 'feta', 'figs', 'filo pastry', 'flour', 'flower sprouts', 'freekeh', 'french beans', 'garlic', 'ginger', "goat's cheese", 'granny smith apples', 'grape', 'green beans', 'greens', 'halloumi', 'halva', 'h